# Preprocessing
### Wiktor Wierzchowski Wojciech Grabias
##### ramka 1

In [ ]:
# W celu uzyskania odpowiednio danych testowych i treningowych należy podmienić nazwy
# W ostatnich komórkach jak i komórce oznaczonej 'Zamiana valid/train/test'

In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import matplotlib.pyplot as plt
from yellowbrick.cluster import KElbowVisualizer
from sklearn_extra.cluster import KMedoids
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AffinityPropagation
from fastDamerauLevenshtein import damerauLevenshtein
from sklearn import metrics
from sklearn.manifold import MDS
from sklearn.cluster import KMeans
import distance
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import re
import webcolors
from tqdm import tqdm
import math
from sklearn.cluster import MiniBatchKMeans

In [61]:
1 - damerauLevenshtein('BOSCH WAN282MXPL', 'BOSCH WAN282MSPL')

0.0625

In [9]:
df = pd.read_csv('pricerunner_aggregate.csv')

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35310 entries, 0 to 35309
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             35310 non-null  int64 
 1   Title          35310 non-null  object
 2   VendorID       35310 non-null  int64 
 3   ClusterID      35310 non-null  int64 
 4   ClusterLabel   35310 non-null  object
 5   CategoryID     35310 non-null  int64 
 6   CategoryLabel  35310 non-null  object
dtypes: int64(4), object(3)
memory usage: 1.9+ MB


Przegląd danych

In [68]:
df['Title'].iloc[10411]

'hewlett packard enterprise amd opteron 8380 2.5ghz 6mb l3 processor'

In [11]:
df.columns = ['ID', 'Title', 'VendorID', 'ClusterID', 'ClusterLabel', 'CategoryID', 'CategoryLabel']

In [12]:
# Są to kolumny jednoznaczne
df.groupby(['CategoryID'])['CategoryLabel'].unique()

CategoryID
2612       [Mobile Phones]
2614                 [TVs]
2615                [CPUs]
2617     [Digital Cameras]
2618          [Microwaves]
2619         [Dishwashers]
2620    [Washing Machines]
2621            [Freezers]
2622     [Fridge Freezers]
2623             [Fridges]
Name: CategoryLabel, dtype: object

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35310 entries, 0 to 35309
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             35310 non-null  int64 
 1   Title          35310 non-null  object
 2   VendorID       35310 non-null  int64 
 3   ClusterID      35310 non-null  int64 
 4   ClusterLabel   35310 non-null  object
 5   CategoryID     35310 non-null  int64 
 6   CategoryLabel  35310 non-null  object
dtypes: int64(4), object(3)
memory usage: 1.9+ MB


Podział zbioru

In [14]:
# Podział ze względu na CategoryLabel
groups = df.groupby('CategoryLabel')
MAIN_df = pd.DataFrame(columns=df.columns)
trash_df = pd.DataFrame(columns=df.columns)
for _, group in groups:
    group_MAIN, group_trash = train_test_split(group, test_size=0.55, random_state=2137)
    MAIN_df = pd.concat([MAIN_df, group_MAIN])
    trash_df = pd.concat([trash_df, group_trash])

# Reset the index of train and test DataFrames
MAIN_df.reset_index(drop=True, inplace=True)
trash_df.reset_index(drop=True, inplace=True)

In [15]:
# Podział ze względu na CategoryLabel
groups = MAIN_df.groupby('CategoryLabel')
TRAIN_df = pd.DataFrame(columns=df.columns)
valid_df = pd.DataFrame(columns=df.columns)
for _, group in groups:
    group_train, group_valid = train_test_split(group, test_size=0.3, random_state=2137)
    TRAIN_df = pd.concat([TRAIN_df, group_train])
    valid_df = pd.concat([valid_df, group_valid])

# Reset the index of train and test DataFrames
TRAIN_df.reset_index(drop=True, inplace=True)
valid_df.reset_index(drop=True, inplace=True)

In [16]:
# Podział ze względu na CategoryLabel
groups = TRAIN_df.groupby('CategoryLabel')
train_df = pd.DataFrame(columns=df.columns)
test_df = pd.DataFrame(columns=df.columns)
for _, group in groups:
    group_train, group_test = train_test_split(group, test_size=0.3, random_state=2137)
    train_df = pd.concat([train_df, group_train])
    test_df = pd.concat([test_df, group_test])

# Reset the index of train and test DataFrames
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [17]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7776 entries, 0 to 7775
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             7776 non-null   object
 1   Title          7776 non-null   object
 2   VendorID       7776 non-null   object
 3   ClusterID      7776 non-null   object
 4   ClusterLabel   7776 non-null   object
 5   CategoryID     7776 non-null   object
 6   CategoryLabel  7776 non-null   object
dtypes: object(7)
memory usage: 425.4+ KB


In [18]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3339 entries, 0 to 3338
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             3339 non-null   object
 1   Title          3339 non-null   object
 2   VendorID       3339 non-null   object
 3   ClusterID      3339 non-null   object
 4   ClusterLabel   3339 non-null   object
 5   CategoryID     3339 non-null   object
 6   CategoryLabel  3339 non-null   object
dtypes: object(7)
memory usage: 182.7+ KB


Obróbka stringów

In [19]:
def preprocess(df):
    def text_correction(text):
        
        #Podstawowa obróbka i usunięcie emoji
        text = re.sub('<[^>]*>', '', text)
        emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
        text = (re.sub('[\W]+', ' ', text.lower()) + ' ' .join(emoticons).replace('-', ''))
        
        # Usunięcie pojedynczych cyfr
        text = re.sub(r'\b\d\b', '', text)
        text = re.sub(r'(?<!\d)\d(?!\d)', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text
    
    # Usunięcie zbędnych kolumn
    df.drop(columns=['ID', 'VendorID', 'ClusterID', 'ClusterLabel', 'CategoryID', 'CategoryLabel'], inplace=True)
    # Korekta tekstu
    df['Title'] = df['Title'].apply(text_correction)
    # Usunięcie informacji o kolorze
    colors = list(webcolors.CSS3_NAMES_TO_HEX)
    df['Title'] = [' '.join([x for x in string.split() if x not in colors]) for string in df['Title']]

In [20]:
preprocess(train_df)
preprocess(test_df)
preprocess(valid_df)

In [21]:
train_df.head()

,Title
0,intel core i 6100 skylake 70ghz dual core
1,intel xeon e 2608lv ghz core 16 threads 20 mb ...
2,intel core i i 8500t hexa core core 10 ghz pro...
3,hp intel xeon x5450 ghz 12mb l processor
4,hewlett packard enterprise 495906 b21 ghz mb l...


Rozbicie stringów z nazwami produktów na listy pojedyńczych słów

In [51]:
# Zamiana valid/train/test
titles_array = valid_df['Title'].values
titles_array

array(['intel xeon bronze 31061 70ghz fc lga1411mb cache box cpu',
       'intel xeon e 4607 processor ghz box 12 mb smart cache',
       'intel xeon e 2660v processor ghz 25 mb smart cache', ...,
       'bosch serie wab28261gb kg load 1400 spin washing machine with acti',
       'siemens wk14d321gb built in kg washer dryer',
       'hotpoint wmbf944k 1400rpm kg experience washing machine'],
      dtype=object)

In [52]:
title_words_array = np.asarray([k.split() for k in titles_array])
title_words_array

array([list(['intel', 'xeon', 'bronze', '31061', '70ghz', 'fc', 'lga1411mb', 'cache', 'box', 'cpu']),
       list(['intel', 'xeon', 'e', '4607', 'processor', 'ghz', 'box', '12', 'mb', 'smart', 'cache']),
       list(['intel', 'xeon', 'e', '2660v', 'processor', 'ghz', '25', 'mb', 'smart', 'cache']),
       ...,
       list(['bosch', 'serie', 'wab28261gb', 'kg', 'load', '1400', 'spin', 'washing', 'machine', 'with', 'acti']),
       list(['siemens', 'wk14d321gb', 'built', 'in', 'kg', 'washer', 'dryer']),
       list(['hotpoint', 'wmbf944k', '1400rpm', 'kg', 'experience', 'washing', 'machine'])],
      dtype=object)

In [36]:
pd.DataFrame(title_words_array, index=None).to_csv('titles_final_valid.csv')

In [37]:
dftest = pd.read_csv('titles_final_valid.csv', index_col=None)

Funkcja tworząca macierz dystansów między nazwami produktów w oparciu o dystans Jaccarda. W przypadku gdy nazwy dzielą jedynie pierwsze słowo jest ono pomijane. Robimy taj ponieważ zauważyliśmy niechciany, duży wpływ nazw firm na wyniki clasteryzacji. Z racji na to że są one z reguły umieszczane w nazwie oferty jako pierwsze taki zabieg ma na celu zminimalizować ten wpływ.

In [38]:
def jacc_sim(words_array, batch_size):
    n = len(words_array)
    jacc_sim = np.zeros((n, n))
    
    with tqdm(total = math.ceil(n/batch_size)**2, desc='Przeliczone batche:') as pbar:
        for i in range(0, n, batch_size):
            batch_start = i
            batch_end = min(n, i + batch_size)

            for j in range(0, n, batch_size):
                if i > j:
                    continue 
                
                batch_start2 = j
                batch_end2 = min(n, j + batch_size)
                
                distances = np.zeros((batch_end-batch_start, batch_end2-batch_start2))
                
                for k in range(batch_end-batch_start):
                    for l in range(batch_end2-batch_start2):
                        d1 = distance.jaccard(title_words_array[i + k], title_words_array[j + l])
                        
                        if(len(title_words_array[i + k])==1 or len(title_words_array[j + l])==1):
                            distances[k][l] = d1
                        else:    
                            d2 = distance.jaccard(title_words_array[i + k][1:], title_words_array[j + l])
                            d3 = distance.jaccard(title_words_array[i + k][1:], title_words_array[j + l][1:])
                            if (d1 != 1 and d2 == 1):
                                distances[k][l] = d3
                            else:
                                distances[k][l] = d1

                jacc_sim[batch_start:batch_end, batch_start2:batch_end2] = distances
                pbar.update(1)
                
                if i < j:
                    jacc_sim[batch_start2:batch_end2, batch_start:batch_end] = distances.transpose()
                    pbar.update(1)

    return jacc_sim

In [53]:
# i love you more
jacc_similarity = jacc_sim(title_words_array, 5000)
# TEMPO: [19:34<00:00, 46.97s/it]

Przeliczone batche:: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:04<00:00, 64.58s/it]


In [54]:
jacc_similarity[0:10, 0:10]

array([[0.        , 0.76470588, 0.82352941, 0.9047619 , 0.95      ,
        0.85      , 0.9       , 0.88888889, 1.        , 0.88888889],
       [0.76470588, 0.        , 0.38461538, 0.8       , 0.77777778,
        0.66666667, 0.72222222, 0.76470588, 0.90909091, 0.83333333],
       [0.82352941, 0.38461538, 0.        , 0.78947368, 0.83333333,
        0.64705882, 0.70588235, 0.75      , 0.9047619 , 0.82352941],
       [0.9047619 , 0.8       , 0.78947368, 0.        , 0.66666667,
        0.86956522, 0.68421053, 0.46666667, 0.63157895, 0.78947368],
       [0.95      , 0.77777778, 0.83333333, 0.66666667, 0.        ,
        0.90909091, 0.78947368, 0.6       , 0.5       , 0.95      ],
       [0.85      , 0.66666667, 0.64705882, 0.86956522, 0.90909091,
        0.        , 0.80952381, 0.85      , 0.91666667, 0.85      ],
       [0.9       , 0.72222222, 0.70588235, 0.68421053, 0.78947368,
        0.80952381, 0.        , 0.70588235, 0.80952381, 0.625     ],
       [0.88888889, 0.76470588, 0.75     

In [55]:
jacc_df = pd.DataFrame(jacc_similarity)
jacc_df.to_csv("jacc_similarity_final_valid.csv")

Analogicznie jak wcześniej tworzymy jeszcze macierz dystansów w oparciu o podobieństwo leksykograficzne przy użyciu odległości damerau-levenshteina. Robimy to z nadzieją na uwzględnienie podobieństwa produktów które różnią się od siebie oznaczeniami technicznymi, np. BOSCH WAN282MXPL i BOSCH WAN282MSPL które funkcja jacc_sim uznała by za zupełnie odległe.

In [42]:
def lex_sim(words_array, batch_size):
    n = len(words_array)
    lex_sim = np.zeros((n, n))
    
    with tqdm(total = math.ceil(n/batch_size)**2, desc='Przeliczone batche:') as pbar:
        for i in range(0, n, batch_size):
            batch_start = i
            batch_end = min(n, i + batch_size)

            for j in range(0, n, batch_size):
                if i > j:
                    continue 
                
                batch_start2 = j
                batch_end2 = min(n, j + batch_size)
                
                distances = np.zeros((batch_end-batch_start, batch_end2-batch_start2))
                
                for k in range(batch_end-batch_start):
                    for l in range(batch_end2-batch_start2):
                        distances[k][l] = 1 - damerauLevenshtein(title_words_array[i + k], title_words_array[j + l], 
                                                            similarity=True)
                        
                lex_sim[batch_start:batch_end, batch_start2:batch_end2] = distances
                pbar.update(1)
                
                if i < j:
                    lex_sim[batch_start2:batch_end2, batch_start:batch_end] = distances.transpose()
                    pbar.update(1)

    return lex_sim

In [43]:
lex_similarity = lex_sim(title_words_array, 5000)

Przeliczone batche:: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:25<00:00, 25.79s/it]


In [44]:
lex_similarity[0:10, 0:10]

array([[0.        , 0.81818182, 0.8       , 0.92857143, 0.90909091,
        0.8       , 0.84615385, 1.        , 1.        , 0.9       ],
       [0.81818182, 0.        , 0.27272727, 0.85714286, 1.        ,
        0.66666667, 0.69230769, 0.90909091, 1.        , 0.90909091],
       [0.8       , 0.27272727, 0.        , 0.78571429, 0.90909091,
        0.66666667, 0.69230769, 0.9       , 0.92857143, 0.9       ],
       [0.92857143, 0.85714286, 0.78571429, 0.        , 0.64285714,
        1.        , 0.71428571, 0.42857143, 0.42857143, 0.71428571],
       [0.90909091, 1.        , 0.90909091, 0.64285714, 0.        ,
        0.93333333, 0.76923077, 0.45454545, 0.5       , 0.90909091],
       [0.8       , 0.66666667, 0.66666667, 1.        , 0.93333333,
        0.        , 0.86666667, 1.        , 0.93333333, 0.93333333],
       [0.84615385, 0.69230769, 0.69230769, 0.71428571, 0.76923077,
        0.86666667, 0.        , 0.76923077, 0.71428571, 0.53846154],
       [1.        , 0.90909091, 0.9      

In [45]:
lex_df = pd.DataFrame(lex_similarity)
lex_df.to_csv("lex_similarity_final_valid.csv")

In [57]:
test_df['Title']

0       intel xeon bronze 31061 70ghz fc lga1411mb cac...
1       intel xeon e 4607 processor ghz box 12 mb smar...
2       intel xeon e 2660v processor ghz 25 mb smart c...
3       hewlett packard enterprise dl380 g intel xeon ...
4       hewlett packard enterprise amd opteron 2427 gh...
                              ...                        
3334           beko wtg821bw kg 1200 spin washing machine
3335       lg f14afdsa 1400 spin kg steam washing machine
3336    bosch serie wab28261gb kg load 1400 spin washi...
3337          siemens wk14d321gb built in kg washer dryer
3338    hotpoint wmbf944k 1400rpm kg experience washin...
Name: Title, Length: 3339, dtype: object